# Data Cleaning and Pre-Processing

In [1]:
#Import required packages and settings

import pandas as pd 
import numpy as np 

import warnings
warnings.filterwarnings("ignore") #Omits warnings from appearing  

# IMPORTANT: Detailed explanation of data sets
This project uses four main data sets to conduct an analysis of international arrival trends to Singapore. 

The first file, **"sg_intl_arrivals.csv"** is part of a larger data set from the Department of Statistics (Singapore), titled "M550001 - International Visitor Arrivals By Inbound Tourism Markets, Monthly". It was exported as a csv file from https://www.tablebuilder.singstat.gov.sg/publicfacing/createDataTable.action?refId=1991 on 12 August 2019. The data set contains international arrival numbers per month from each country, between 1978 to 2019 (Jun). 

The second file, **"gdp_per_capita.csv"** is part of a larger data set from the World Bank Group. This dataset contains annual GDP per capita (in constant 2010 US$) of 264 countries/territories/groups, between 1960 to 2018. It was exported as a csv file from https://data.worldbank.org/indicator/NY.GDP.PCAP.KD on 12 August 2019.

The third file, **"country_population.csv"** is part of a larger dataset from the World Bank Group. This dataset contains annual population totals of 264 countries/territories/groups, between 1960 to 2018. It was exported as a csv file from https://data.worldbank.org/indicator/SP.POP.TOTL?view=chart on 12 August 2019.

The fourth file, **"sgp_dist_lang.csv"** is part of a larger dataset from the CEPII (French: Institute for Research on the International Economy), titled "dist_cepii". This data set provides bilateral data, such as distance between countries (in km), and dummy variables indicating if two countries share a common language or a colonial relationship. For our purposes, we are interested in the following variables:

* Country codes
* Common language official (0 or 1): whether a particular country shares a commmon official language with Singapore. An official language is defined as one spoken by at least 20% of the population in the country (for Singapore it is English and Chinese).
* Common language ethno (0 or 1): whether a particular country shares another spoken language with Singapore. This is defined as one spoken by between 9-20% of the population the country (for Singapore it is English and Malay)
* Weighted distance: distance between two countries based on bilateral distances between their largest cities, weighted by city population over the country's population.

For our analysis, we will define a country as sharing a common language with Singapore, as long as either common language official or common language ethno is 1. 

The data sets and accompanying notes were downloaded from http://www.cepii.fr/CEPII/en/bdd_modele/presentation.asp?id=6 on 12 August 2019.

# Data Import, Merging & Exploration
This Notebook covers the importing, merging and cleaning of mulitple datasets for our analysis. We will be merging the (1) sg_intl_arrivals.csv, (2) gdp_per_capita.csv, (3) country_population.csv and (4) sgp_dist_lang.csv data sets into one.  

We then proceed to obtain general information about the merged data set, such as its shape and summary statistics. Once we are satisfied with the merged dataset, we will export it for further analysis. 

## 1.1 Import and tidy data

In [2]:
#Import the Singapore International Arrivals (2000 -2017) dataset and display first 5 rows 

df_arrivals = pd.read_csv('sg_intl_arrivals.csv')
df_arrivals.head()

,month_year,region,country,visitor_count
0,2000-01,Southeast Asia,Brunei Darussalam,4138
1,2000-01,Southeast Asia,Indonesia,149528
2,2000-01,Southeast Asia,Malaysia,41261
3,2000-01,Southeast Asia,Myanmar,1479
4,2000-01,Southeast Asia,Philippines,11809


In [3]:
#Import the weighted distance and common language dataset and display first 5 rows 

df_distlang = pd.read_csv('sgp_dist_lang.csv')
df_distlang.head()

,country_code,comm_lang,distw_km
0,ABW,1,18355.23
1,AFG,0,5248.137
2,AGO,0,10046.3
3,AIA,1,17433.05
4,ALB,0,9420.047


In [4]:
#Import the GDP per capita (in 2010 US$) dataset and display first 5 rows 

df_gdp = pd.read_csv('gdp_per_capita.csv')
df_gdp.head() 

,country,country_code,2000,2001,2002,2003,2004,2005,2006,2007,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,28417.384210,26966.054790,25508.302520,25469.287410,27005.529530,26979.885360,27046.760420,27428.120200,...,24464.174520,23512.602600,24231.338900,23777.316050,24629.079980,24692.497180,24452.606570,24288.987100,24508.809140,NaN
1,Afghanistan,AFG,NaN,NaN,339.633338,352.243953,341.612456,365.548734,372.896735,412.919585,...,495.108928,550.514974,536.012486,584.907376,597.525224,594.574110,585.708288,583.055107,583.869634,563.825663
2,Angola,AGO,2189.560753,2208.791536,2426.431783,2412.392521,2582.646476,2866.434694,3085.424833,3394.512335,...,3544.026552,3585.905553,3580.269910,3750.209097,3799.429617,3846.240911,3751.694465,3533.865160,3413.656438,3229.619740
3,Albania,ALB,2244.564836,2453.557464,2572.652201,2725.096745,2887.291291,3062.592504,3263.812400,3485.227628,...,3928.342143,4094.360204,4210.077005,4276.917643,4327.608232,4413.335122,4524.467508,4683.519217,4867.632465,5075.354985
4,Andorra,AND,40801.542130,41420.846180,42396.302400,45519.492380,47032.866890,48831.929360,49708.400490,48710.664620,...,41979.865900,39736.354060,38205.771090,38190.586590,39104.301850,40785.049230,41765.920350,42958.558390,43942.938190,44569.783010


In [5]:
#Convert the GDP per capita dataframe into tidy data format 

list_years = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']
df_gdp_tidy = pd.melt(frame=df_gdp,id_vars=['country','country_code'], value_vars=list_years,var_name='year',value_name='gdp_per_capita')
df_gdp_tidy.head(5)

,country,country_code,year,gdp_per_capita
0,Aruba,ABW,2000,28417.384210
1,Afghanistan,AFG,2000,NaN
2,Angola,AGO,2000,2189.560753
3,Albania,ALB,2000,2244.564836
4,Andorra,AND,2000,40801.542130


In [6]:
#Import the country population dataset and display first 5 rows 

df_pop = pd.read_csv('country_population.csv')
df_pop.head()

,country,country_code,2000,2001,2002,2003,2004,2005,2006,2007,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,90853.0,92898.0,94992.0,97017.0,98737.0,100031.0,100832.0,101220.0,...,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0,105264.0,105845.0
1,Afghanistan,AFG,20093756.0,20966463.0,21979923.0,23064851.0,24118979.0,25070798.0,25893450.0,26616792.0,...,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0,35530081.0,37172386.0
2,Angola,AGO,16440924.0,16983266.0,17572649.0,18203369.0,18865716.0,19552542.0,20262399.0,20997687.0,...,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0,29784193.0,30809762.0
3,Albania,ALB,3089027.0,3060173.0,3051010.0,3039616.0,3026939.0,3011487.0,2992547.0,2970017.0,...,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0
4,Andorra,AND,65390.0,67341.0,70049.0,73182.0,76244.0,78867.0,80991.0,82683.0,...,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281.0,76965.0,77006.0


In [7]:
#Convert the country population dataframe into tidy data format 

list_years = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']
df_pop_tidy = pd.melt(frame=df_pop,id_vars=['country','country_code'], value_vars=list_years,var_name='year',value_name='total_pop')
df_pop_tidy.head()

,country,country_code,year,total_pop
0,Aruba,ABW,2000,90853.0
1,Afghanistan,AFG,2000,20093756.0
2,Angola,AGO,2000,16440924.0
3,Albania,ALB,2000,3089027.0
4,Andorra,AND,2000,65390.0


## 1.2 Restructure the datasets by year 
By the end of this section, we will have a single merged dataset containing the information seen above. 

In [8]:
#Split the 'month-year' column of the Arrivals dataset into 'year' and 'month' respectively 

df_arrivals['year'], df_arrivals['month'] = df_arrivals['month_year'].str.split('-', 1).str
df_arrivals.head()

,month_year,region,country,visitor_count,year,month
0,2000-01,Southeast Asia,Brunei Darussalam,4138,2000,01
1,2000-01,Southeast Asia,Indonesia,149528,2000,01
2,2000-01,Southeast Asia,Malaysia,41261,2000,01
3,2000-01,Southeast Asia,Myanmar,1479,2000,01
4,2000-01,Southeast Asia,Philippines,11809,2000,01


In [9]:
#Visitor count was in string, and changed to numeric for computation

df_arrivals['visitor_count'] = pd.to_numeric(df_arrivals['visitor_count'],errors='coerce')

In [10]:
#Sum visitor arrivals by country and year

df_arrivals = df_arrivals.groupby(['country', 'year'])['visitor_count'].sum().reset_index()

In [11]:
#Merge the arrivals data with population, gdp, distance and language

df_merged = pd.merge(df_arrivals, df_pop_tidy, how='inner',on = ['country','year'])
df_merged = pd.merge(df_merged, df_gdp_tidy, how='inner',on = ['country','year','country_code'])
df_merged = pd.merge(df_merged, df_distlang, how='inner',on = 'country_code')
df_merged

,country,year,visitor_count,country_code,total_pop,gdp_per_capita,comm_lang,distw_km
0,Australia,2000,510347.0,AUS,19153000.0,44313.318230,1,5893.497
1,Australia,2001,550681.0,AUS,19413000.0,44564.976560,1,5893.497
2,Australia,2002,538378.0,AUS,19651400.0,45786.642810,1,5893.497
3,Australia,2003,392891.0,AUS,19895400.0,46575.415330,1,5893.497
4,Australia,2004,561219.0,AUS,20127400.0,47880.611660,1,5893.497
5,Australia,2005,620196.0,AUS,20394800.0,48760.355170,1,5893.497
6,Australia,2006,691632.0,AUS,20697900.0,49408.052740,1,5893.497
7,Australia,2007,768490.0,AUS,20827600.0,50955.055950,1,5893.497
8,Australia,2008,833156.0,AUS,21249200.0,51770.907150,1,5893.497
9,Australia,2009,830299.0,AUS,21691700.0,51689.913560,1,5893.497


As the original arrivals dataset is split into months, we have to sum by country and year to get annual visitor counts for each country. Afterwards, we merge the data on annual visitor arrivals, annual GDP, total population, whether the country shares a common language with Singapore and its weighted distance from Singapore. 

In [12]:
#Export the merged dataframe for further analysis

df_merged.to_csv('df_merged.csv')